In [3]:
#GETTING MY ID:
import json

id = open('my_id.json', 'r')
mi = json.load(id)

from azureml.core import Workspace

ws = Workspace.create(name="testmodel",
                      subscription_id = mi["my_id"],
                      resource_group = "deploymodel",
                      location = "centralindia") #centralindia

Performing interactive authentication. Please follow the instructions on the terminal.


AuthenticationException: AuthenticationException:
	Message: Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000015FAB4F2450>: Failed to resolve 'login.microsoftonline.com' ([Errno 11001] getaddrinfo failed)"))
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NameResolutionError(\"<urllib3.connection.HTTPSConnection object at 0x0000015FAB4F2450>: Failed to resolve 'login.microsoftonline.com' ([Errno 11001] getaddri

In [4]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="classifier",
                                  workspace=ws)

Registering model classifier


In [5]:
from azureml.core.environment import Environment
virtual_env = Environment("env")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [6]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='testmodel',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Felip\AppData\Local\Temp\ipykernel_32672\1570049851.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [7]:
service.wait_for_deployment()

Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 72659181-c870-4516-8e74-c7d0cd464c93
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'fd4d943c-c7c7-4828-ae43-32cb497576d6' with object id 'fd4d943c-c7c7-4828-ae43-32cb497576d6' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/744cc732-ce55-491a-8d06-aa9c493a4b37/resourceGroups/actividad1/providers/Microsoft.ContainerInstance/containerGroups/testmodel-m1C8qs7rA06nZ5Kq_Jgf-Q' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 72659181-c870-4516-8e74-c7d0cd464c93
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'fd4d943c-c7c7-4828-ae43-32cb497576d6' with object id 'fd4d943c-c7c7-4828-ae43-32cb497576d6' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/744cc732-ce55-491a-8d06-aa9c493a4b37/resourceGroups/actividad1/providers/Microsoft.ContainerInstance/containerGroups/testmodel-m1C8qs7rA06nZ5Kq_Jgf-Q' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service s

In [11]:
scoring_uri = service.scoring_uri
# Save the uri to a file
with open('scoring_uri.txt', 'w') as f:
    f.write(scoring_uri)


In [2]:
import pandas as pd
import json
input_data = pd.read_csv("load_data/temp_data/output.csv")
    # Serialize the first 5 rows of the DataFrame to JSON
input_data = input_data.head(5).to_json(orient="records")
input_data = {
    "data": json.loads(input_data)  # Asegúrate de que sea una lista de registros
}

model_input = json.dumps(input_data)

headers = {"Content-Type": "application/json"}
# ===================AQUI VA EL URI==================
scoring_uri = open('scoring_uri.txt', 'r').readline().strip()
# ====================AQUI VA EL URI==================
import requests
response = requests.post(scoring_uri, 
                         data=model_input, 
                         headers=headers)

if response.status_code == 200:
  result = json.loads(response.json())
  print(result)
  input_data["median_house_value"] = result["result"]
  display(pd.DataFrame(input_data["data"]))
else:
  print(f"Error: {response.text}")

FileNotFoundError: [Errno 2] No such file or directory: 'load_data/temp_data/output.csv'